# Extraindo Dados

Desenvolvendo uma tabela que contenha os seguintes dados: 
- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- os sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o numero de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

Importando o Pandas e o Csv.

In [272]:
import pandas as pd
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [273]:
df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

Listando as colunas do dataframe.

In [274]:
df_dados.columns

Index(['discente', 'unidade', 'media_final', 'descricao', 'ano',
       'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status'],
      dtype='object')

# Filtros

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [275]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

Listando os dados NaN.

In [276]:
df_dados_filtrado.isnull().sum()

discente             0
unidade           2034
media_final       4449
descricao            0
ano                  0
id_componente        0
nome                 0
ch_total             0
sexo                 0
ano_nascimento       0
ano_ingresso         0
status               0
dtype: int64

Preenchendo os dados NaN da coluna **unidade** com 1.

In [277]:
df_dados_filtrado.loc[:, 'unidade'] = df_dados_filtrado['unidade'].fillna(1)

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [278]:
df_dados_filtrado = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Renomeando dados da coluna descrição.

In [279]:
df_dados_filtrado = df_dados_filtrado.replace({'descricao':
               {'APROVADO POR NOTA':'APROVADO',
                'REPROVADO POR FALTAS':'REPROVADO',
                'REPROVADO POR MÉDIA E POR FALTAS':'REPROVADO',
                'REPROVADO POR NOTA E FALTA':'REPROVADO',
                'REPROVADO POR NOTA':'REPROVADO',
                }}, regex=True)

Contando os valores dos dados da coluna descrição.

In [280]:
df_dados_filtrado.descricao.value_counts()

descricao
APROVADO       8390
REPROVADO      2771
EXCLUIDA       1118
CANCELADO       898
TRANCADO        591
INDEFERIDO      397
DESISTENCIA     178
Name: count, dtype: int64

## Extraindo dados

### Colunas com as Disciplinas

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [281]:
quantidade_disciplinas = df_dados_filtrado.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [282]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

Substituindo NaN por 0 nas colunas de nome.

In [283]:
tabela_final = tabela_final.fillna(0)

### Coluna com o Ano de Ingresso

Adicionando o *ano_ingresso* para cada discente.

In [284]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o Sexo do Discente

Definir a opção global para permitir o downcasting silencioso

In [285]:
pd.set_option('future.no_silent_downcasting', True)

Alterando o conteúdo da coluna *sexo*.

In [286]:
df_dados_filtrado['sexo'] = df_dados_filtrado['sexo'].replace({'M': 1, 'F': 0})
df_dados_filtrado = df_dados_filtrado.infer_objects(copy=False)

Adicionando a coluna *sexo*.

In [287]:
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'sexo']]
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')

### Coluna com o Status do Discente

Alterando o conteúdo da coluna *status*.

In [288]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({'CANCELADO': -1, 'ATIVO': -1, 'ATIVO - FORMANDO': -1,'FORMADO': 1,'CONCLUÍDO': 1})

Adicionando o *status* para cada discente.

In [289]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')

### Coluna com a Carga Horária Cumprida

Agrupando por discente e somando a *carga horária*.

In [290]:
df_carga_horaria_cumprida = df_dados_filtrado.groupby('discente')['ch_total'].sum().reset_index()
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)

Adicionando a *carga horária cumprida* de cada discente.

In [291]:
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [292]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com o Semestre Cursado

Identificar *semestres* únicos cursados por cada discente.

In [293]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [294]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [295]:
tabela_final['semestre_dividido'] = tabela_final['semestre'] / 8

### Coluna com o Último Período Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [296]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [297]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente se Matriculou

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [298]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [299]:
tabela_final['matricula'] = matricula

### Coluna com o Tempo Relativo que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [300]:
tabela_final['ch_cumprida_dividida'] = pd.to_numeric(tabela_final['ch_cumprida_dividida'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [301]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [302]:
tabela_final['tempo_relativo'] = tabela_final['ch_cumprida_dividida'] * tabela_final['status']

### Coluna com a quantidade de vezes que o discente foi Aprovado

Listar todas as disciplinas.

In [303]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [304]:
aprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'] == 'APROVADO']

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [305]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [306]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [307]:
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi Reprovado

Listar todas as disciplinas.

In [308]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações*.

In [309]:
reprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'] == 'REPROVADO']

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [310]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [311]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

Mesclar as *reprovações* de volta ao DataFrame original.

In [312]:
df_final_reprovacao = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final_reprovacao, on='discente', how='left')

Mudando a ordem das colunas.

In [313]:
colunas_ordenadas = ['discente', 'sexo', 'ano_ingresso', 'ultimo_periodo', 'semestre',
                    'semestre_dividido', 'ch_cumprida', 'ch_cumprida_dividida', 
                    'tempo_relativo', 'status', 'matricula',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'LÓGICA',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'ÉTICA',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO']


Reordenando as colunas.

In [314]:
tabela_final = tabela_final[colunas_ordenadas]
tabela_final

,discente,sexo,ano_ingresso,ultimo_periodo,semestre,semestre_dividido,ch_cumprida,ch_cumprida_dividida,tempo_relativo,status,...,PROGRAMAÇÃO VISUAL_REPROVADO,MATEMÁTICA FINANCEIRA,MATEMÁTICA FINANCEIRA_APROVADO,MATEMÁTICA FINANCEIRA_REPROVADO,SISTEMAS DE APOIO À DECISÃO,SISTEMAS DE APOIO À DECISÃO_APROVADO,SISTEMAS DE APOIO À DECISÃO_REPROVADO,ÉTICA,ÉTICA_APROVADO,ÉTICA_REPROVADO
0,001cea3c82e2010681f2cdeab21e5ecf,1,2018,20181,1,0.125,330,0.180328,-0.180328,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,005c14d7c07bf7980b60c703f99c5ee7,1,2018,20221,10,1.250,2340,1.278689,-1.278689,-1,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0107fd69d8cd7e3d30dede96fb68bfe5,1,2011,20121,3,0.375,870,0.475410,-0.475410,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,014789363f7940922e71e710ee9d22bc,1,2016,20206,11,1.375,2430,1.327869,1.327869,1,...,0.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
4,014f0dec46fe7a9c5836527662e1df10,1,2020,20206,3,0.375,630,0.344262,-0.344262,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,fe802d8d85de6f842749468401d1146c,1,2022,20222,2,0.250,540,0.295082,-0.295082,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
678,fe87dfa176a74fc10a5cb701b9fb5dd4,1,2016,20206,3,0.375,420,0.229508,0.229508,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
679,fec9ed6026d55ecdf514c640312c3d08,1,2020,20222,7,0.875,1470,0.803279,-0.803279,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
680,ff56f2c5048dae0797fd3e851572b80c,1,2014,20192,12,1.500,3390,1.852459,1.852459,1,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


Para salvar o DataFrame no formato CSV com a separação por ';' e garantir que os dados numéricos estejam no formato correto.

In [315]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=';', quoting=csv.QUOTE_NONNUMERIC)